In [34]:
#include<mlpack/xeus-cling.hpp>
#include<mlpack/core.hpp>
#include<mlpack/core/data/split_data.hpp>
#include<mlpack/methods/ann/layer/layer.hpp>
#include<mlpack/methods/ann/ffn.hpp>
#include<bits/stdc++.h>

In [2]:
#include<ensmallen.hpp>

In [19]:
using namespace mlpack;
using namespace mlpack::ann;
using namespace arma;
using namespace std;
using namespace ens;

In [4]:
arma::Row<size_t> getLabels(arma::mat predOut){
    arma::Row<size_t> predLabels(predOut.n_cols);
    for( arma::uword i = 0; i < predOut.n_cols; ++i){
        predLabels(i) = predOut.col(i).index_max();
    }
    return predLabels;
}

In [5]:
constexpr double RATIO = 0.1;
constexpr int MAX_ITERATIONS = 0;
constexpr double STEP_SIZE = 1.2e-3;
constexpr int BATCH_SIZE = 50;

In [7]:
arma::mat dataset;

In [9]:
data::Load("/home/viole/swaingotnochill/examples/generating_hand_written_digits_mnist_with_gan/digit-recognizer/train.csv", dataset, true);

In [11]:
arma::mat train, valid;

In [12]:
data::Split(dataset, train, valid, RATIO);

In [15]:
const arma::mat trainX = train.submat(1, 0, train.n_rows - 1, train.n_cols - 1);
const arma::mat validX = valid.submat(1, 0, valid.n_rows - 1, valid.n_cols - 1);

In [17]:
const arma::mat trainY = train.row(0);
const arma::mat validY = valid.row(0);

In [28]:
using namespace mlpack::ann;
FFN<NegativeLogLikelihood<>, RandomInitialization> model;

In [39]:
model.Add<mlpack::ann::Convolution<>> (1,6,5,5,1,1,0,0,28,28);
                        

In file included from input_line_7:5:
/home/viole/anaconda3/envs/notebook/include/mlpack/methods/ann/ffn.hpp:290:36: error: no member named 'push_back' in
      'std::vector<boost::variant<mlpack::ann::AdaptiveMaxPooling<arma::Mat<double>,
      arma::Mat<double> > *,
      mlpack::ann::AdaptiveMeanPooling<arma::Mat<double>,
      arma::Mat<double> > *, mlpack::ann::Add<arma::Mat<double>,
      arma::Mat<double> > *, mlpack::ann::AddMerge<arma::Mat<double>,
      arma::Mat<double>> *, mlpack::ann::AlphaDropout<arma::Mat<double>,
      arma::Mat<double> > *,
      mlpack::ann::AtrousConvolution<mlpack::ann::NaiveConvolution<mlpack::ann::ValidConvolution>,
      mlpack::ann::NaiveConvolution<mlpack::ann::FullConvolution>,
      mlpack::ann::NaiveConvolution<mlpack::ann::ValidConvolution>,
      arma::Mat<double>, arma::Mat<double> > *,
      mlpack::ann::BaseLayer<mlpack::ann::LogisticFunction,
      arma::Mat<double>, arma::Mat<double> > *,
      mlpack::ann::BaseLayer<mlpack::ann::Iden

      mlpack::ann::CReLU<arma::Mat<double>, arma::Mat<double> > *,
      mlpack::ann::DropConnect<arma::Mat<double>, arma::Mat<double> > *,
      mlpack::ann::Dropout<arma::Mat<double>, arma::Mat<double> > *,
      mlpack::ann::ELU<arma::Mat<double>, arma::Mat<double> > *,
      mlpack::ann::FastLSTM<arma::Mat<double>, arma::Mat<double> > *,
      mlpack::ann::FlexibleReLU<arma::Mat<double>, arma::Mat<double> > *,
      mlpack::ann::GRU<arma::Mat<double>, arma::Mat<double> > *,
      mlpack::ann::HardTanH<arma::Mat<double>, arma::Mat<double> > *,
      mlpack::ann::Join<arma::Mat<double>, arma::Mat<double> > *,
      mlpack::ann::LayerNorm<arma::Mat<double>, arma::Mat<double> > *,
      mlpack::ann::LeakyReLU<arma::Mat<double>, arma::Mat<double> > *,
      mlpack::ann::Linear<arma::Mat<double>, arma::Mat<double>,
      mlpack::ann::NoRegularizer> *,
      mlpack::ann::LinearNoBias<arma::Mat<double>, arma::Mat<double>,
      mlpack::ann::NoRegularizer> *,
      mlpack::ann::LogSoftMax<a

Interpreter Error: 